In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Create Tic-Tac-Toe smart player with fully connected dense layers

Yeah, it's a little bit overkill, but i thought that it would be nice task to practice and have fun ;)

In [2]:
"""Check winner"""
def check_board(board):
    win_x = np.array([1, 1, 1])
    win_o = np.array([-1, -1, -1])
    for _ in range(4):
        if (board[0] == win_x).all() or (board[1] == win_x).all() or ([board[0][0], board[1][1], board[2][2]] == win_x).all():
            return 1
        if (board[0] == win_o).all() or (board[1] == win_o).all() or ([board[0][0], board[1][1], board[2][2]] == win_o).all():
            return -1
        
        board = np.rot90(board)
    return 0

In [3]:
"""
Create random games to make dataset
"""
import copy
def crate_dataset(size):
    dataset = []
    played = 0
    while played < size:
        game = []
        board = np.zeros(shape = (3,3))
        [turn] = np.random.randint(0, 2, 1)
        turn = -1 if turn == 0 else 1
        rows, cols = np.where(board == 0)
        possible_moves = np.concatenate((rows.reshape(-1,1), cols.reshape(-1,1)), axis = 1)
        while possible_moves.shape[0] != 0:
            [move] = np.random.randint(0, possible_moves.shape[0], 1)
            board[possible_moves[move][0]][possible_moves[move][1]] = turn
            turn = -turn 
            rows, cols = np.where(board == 0)
            possible_moves = np.concatenate((rows.reshape(-1,1), cols.reshape(-1,1)), axis = 1)
            game.append(board.astype(np.int32).tolist())
            winner = check_board(copy.deepcopy(board))
            if winner != 0:
                break
        if game not in dataset:
            played += 1
            dataset.append([(line, winner) for line in game])
    final_moves = []
    final_target = []
    for game in dataset:
        for move in game:
            final_moves.append(move[0])
            final_target.append(move[1])
    random = np.random.permutation(range(np.shape(final_moves)[0]))
    return np.array(final_moves)[random.astype(int)], np.array(final_target)[random.astype(int)].reshape(-1,1), dataset

In [4]:
X, y, game_dataset = crate_dataset(2000)

In [ ]:
size = np.shape(X)[0] * 80 // 100

In [ ]:
train = tf.data.Dataset.from_tensor_slices((X[:size], y[:size] + [1])).shuffle(800).batch(32).prefetch(1)
valid = tf.data.Dataset.from_tensor_slices((X[size:], y[size:] + [1])).batch(np.shape(X[size:])[0]).prefetch(1)

In [ ]:
smart_player = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[3,3]),
    keras.layers.Dense(450, activation = 'selu', input_shape = [9], 
                       kernel_constraint=keras.constraints.max_norm(2.)),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(300, activation = 'selu', 
                       kernel_constraint=keras.constraints.max_norm(2.)),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(150, activation = 'selu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(50, activation = 'selu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(20, activation = 'selu'),
    keras.layers.Dense(3, activation = 'softmax')
])

In [275]:
smart_player.compile(loss = 'sparse_categorical_crossentropy', 
                     optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, nesterov = True),
                     metrics = ['accuracy'])

In [276]:
early_stopping = keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)

In [277]:
history = smart_player.fit(train, epochs = 100, validation_data = valid, callbacks = [early_stopping])

Epoch 1/100
383/383 [==============================] - 3s 8ms/step - loss: 0.9852 - accuracy: 0.5165 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
383/383 [==============================] - 2s 5ms/step - loss: 0.9214 - accuracy: 0.5551 - val_loss: 0.8677 - val_accuracy: 0.5755
Epoch 3/100
383/383 [==============================] - 2s 5ms/step - loss: 0.9137 - accuracy: 0.5526 - val_loss: 0.8770 - val_accuracy: 0.5752
Epoch 4/100
383/383 [==============================] - 2s 5ms/step - loss: 0.9039 - accuracy: 0.5621 - val_loss: 0.8706 - val_accuracy: 0.5818
Epoch 5/100
383/383 [==============================] - 2s 5ms/step - loss: 0.8956 - accuracy: 0.5706 - val_loss: 0.8608 - val_accuracy: 0.5821
Epoch 6/100
383/383 [==============================] - 2s 5ms/step - loss: 0.8947 - accuracy: 0.5654 - val_loss: 0.8541 - val_accuracy: 0.5883
Epoch 7/100
383/383 [==============================] - 2s 5ms/step - loss: 0.8915 - accuracy: 0.5634 - val_loss: 0.8705 - val_accuracy

In [279]:
smart_player.save('smart_player2.h5')

In [6]:
smart_player =  tf.keras.models.load_model('smart_player2.h5')

In [7]:
"""Random player"""
def random_player(board, turn, _):
    rows, cols = np.where(board == 0)
    possible_moves = np.concatenate((rows.reshape(-1,1), cols.reshape(-1,1)), axis = 1)
    [move] = np.random.randint(0, possible_moves.shape[0], 1)
    board[possible_moves[move][0]][possible_moves[move][1]] = turn

In [8]:
"""Smart player using pretrained model"""
def smart_player_func(board, turn, model):
    rows, cols = np.where(board == 0)
    possible_moves = np.concatenate((rows.reshape(-1,1), cols.reshape(-1,1)), axis = 1)
    proba = []
    
    for i in range(len(possible_moves)):
        temp_board = np.array([copy.deepcopy(board).tolist()])
        temp_board[0][possible_moves[i][0]][possible_moves[i][1]] = turn
        proba.append(model.predict_proba(temp_board))
    proba = np.vstack(proba)
    formula = proba[:, turn+1] - proba[:, -turn+1]
    best = np.argmax(formula)
    if formula[best] < 0.13:
        best = np.argmax(proba[:, 1])
    board[possible_moves[best][0]][possible_moves[best][1]] = turn
#smart_player_func(np.zeros(shape = (3,3)), -1, smart_player)

In [14]:
from collections import Counter
def game(rounds = 1, turn = None, smart = 1, player1 = smart_player_func, player2 = random_player, model = smart_player):
    score_board = []
    for _ in range(rounds):
        board = np.zeros(shape = (3,3))
        
        if turn is None:
            turn = -1 if np.random.randint(0,2) == 0 else 1
        
        winer = 0
        rows, cols = np.where(board == 0)
        possible_moves = np.concatenate((rows.reshape(-1,1), cols.reshape(-1,1)), axis = 1)
        while possible_moves.shape[0] != 0:
            
            if turn == smart:
                player1(board, turn, model)
            else:
                player2(board, turn, model)
            
            winner = check_board(copy.deepcopy(board))
            if winner != 0:
                break
            turn = -turn
            rows, cols = np.where(board == 0)
            possible_moves = np.concatenate((rows.reshape(-1,1), cols.reshape(-1,1)), axis = 1)
            
        score_board.append(winner)
    score = Counter(score_board)
    print('Win: '+ str(np.round(score[smart] / rounds * 100, 2)) + '%')
    print('Lose: '+ str(np.round(score[-smart] / rounds  * 100, 2)) + '%')
    print('Defeat: '+ str(np.round(score[0] / rounds * 100, 2)) + '%')
    

In [15]:
game(rounds = 500, turn = 1, smart = 1, model = smart_player)

Win: 97.6%
Lose: 1.4%
Defeat: 1.0%


In [16]:
game(rounds = 500, turn = -1, smart = 1, model = smart_player)

Win: 98.4%
Lose: 0.6%
Defeat: 1.0%
